## This is the baseline model, it predicts the wind farm dependent average electricity production and aggregate predictions overall wind farms

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data = pd.read_csv('../data/GEFCom2014Data/Wind/raw_data_incl_features.csv', parse_dates=['TIMESTAMP'])
data.dropna(inplace=True)
RSEED = 42

In [ ]:
# train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RSEED, shuffle=True)

In [ ]:
# zones to loop over
zones = np.sort(train.ZONEID.unique()) 

# baseline predictions of all sites will be merged into one DataFrame to calculate the RMSE with respect to the observations of all zones
# finalpred = pd.DataFrame() 
df_results = pd.DataFrame(index = [f'ZONE{zone}' for zone in zones] + ['TOTAL'], columns = ['BEST_PARAMS','CV','MODEL','FC','TESTSCORE','TRAINSCORE'])
df_results.loc['TOTAL'].TRAINSCORE = 0
df_results.loc['TOTAL'].TESTSCORE = 0
df_results['MODEL'] = 'Baseline'

# loop over all zones
for zone in zones:

    # get train and test data of individual zones
    train = train[train.ZONEID == zone].TARGETVAR
    ytest =  test[test.ZONEID == zone].TARGETVAR

    # baseline predicton for individual zone
    pred_train = np.ones(len(ytrain)) * np.mean(ytrain)
    pred_test = np.ones(len(ytest)) * np.mean(ytrain)

    df_results.loc[f'ZONE{zone}'].TRAINSCORE = mean_squared_error(ytrain, pred_train, squared=False)
    df_results.loc[f'ZONE{zone}'].TESTSCORE = mean_squared_error(ytest, pred_test, squared=False)

    df_results.loc['TOTAL'].TRAINSCORE += np.power(df_results.loc[f'ZONE{zone}'].TRAINSCORE,2) * len(ytrain)/len(train)
    df_results.loc['TOTAL'].TESTSCORE += np.power(df_results.loc[f'ZONE{zone}'].TESTSCORE,2) * len(ytest)/len(test)

    df_results.loc['TOTAL'].TRAINSCORE = np.power(df_results.loc['TOTAL'].TRAINSCORE,.5)
    df_results.loc['TOTAL'].TESTSCORE = np.power(df_results.loc['TOTAL'].TESTSCORE,.5)

    df_results.index.set_names(['ZONE'], inplace=True)
